In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

c:\Users\PEDRO\OneDrive\Documentos\Insper\2025.1\Insper Data\AECOM\Projeto_InsperData_AECOM\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
testes = pd.read_excel('../docs/testes.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'docs/testes.xlsx'

acesso negado

In [13]:
def captura_num_processos(filtrados):
    copia_filtrados = filtrados.copy()
    sucessos = 0
    processos = []

    for i in range(copia_filtrados.shape[0]):
        link = copia_filtrados.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        num_processo = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            num_processo = match.group(0)
            sucessos += 1

        processos.append(num_processo)
    
    print(f"Números de processos capturados: {sucessos}")

    copia_filtrados["num_processo"] = processos
    return copia_filtrados
    
filtrados_com_num = captura_num_processos(testes)
# filtrados_com_num.to_excel("processos_fitrados.xlsx")
filtrados_com_num

Números de processos capturados: 100


,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
...,...,...,...,...,...,...,...,...,...,...,...
98,8001732-18.2024.8.05.0261,641758157,1542934097,http://jud-anexos.digesto.com.br/a1b0cc6140bfa...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - ALVARADELEVANTAMENTO,8001732-18.2024.8.05.0261
99,8001732-18.2024.8.05.0261,641758157,1542934101,http://jud-anexos.digesto.com.br/82219c92d2729...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - BANCOITAUCONSIGNADOSA,8001732-18.2024.8.05.0261
100,8001732-18.2024.8.05.0261,641758157,1542934102,http://jud-anexos.digesto.com.br/1be43c6232a87...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - 2SENTENCA8000534142...,8001732-18.2024.8.05.0261
101,8001732-18.2024.8.05.0261,641758157,1542934104,http://jud-anexos.digesto.com.br/483fc8cddfa24...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - 3LAUDOCUMPRIMENTOOBF,8001732-18.2024.8.05.0261


In [14]:
processos_nao_encontrados = filtrados_com_num.loc[filtrados_com_num['num_processo'] == "Não encontrado", :]

In [16]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = processos_nao_encontrados[processos_nao_encontrados['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)

In [17]:
acessos_negados

,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo


Dentre os testes, nenhum acesso negado

# Utilização de Gemini 

Testar a validade de uso do Gemini a partir de uma amostra de testes da base de dados do jusbrasil.

In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = testes
response = requests.get(filtrados.iloc[1,3])
response.encoding = 'utf-8'
content_type = response.headers.get('Content-Type', '')
texto_extraido = ""

if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(response.content)
        tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
    with pymupdf.open(tmp_path) as doc:
        for page in doc:
            texto_extraido += page.get_text()
    
elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
    soup = BeautifulSoup(response.content, 'html.parser')
    texto_extraido = soup.get_text()

# Prompt
prompt = f"""Você está recebendo um texto bruto de um documento sobre
        algum processo judiciário. O seu interesse principal é descobrir se este texto trata de um caso relacionado à 
        valoração de danos ambientais.
        Assim, a partir das informações nele contidas, devolva APENAS UMA LISTA (NÃO DEVOLVA O TEXTO) 
        com as seguintes informações em ordem:
        Elemento 0 da lista: Texto fala de um processo em que houve algum tipo de dano ambiental (True ou False)
        Elemento 1 da lista: Texto menciona uma georreferência do local afetado pelo dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 2 da lista: A UF do local ou seja "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará",
    "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará",
    "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro",
    "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima",
    "Santa Catarina", "São Paulo", "Sergipe", "Tocantins" ou variações (devolva a string
        "Não encontrado" caso não exista)
        Elemento 3 da lista: O município ou cidade do local do processo (devolva a string
        "Não encontrado" caso não exista). 
        Elemento 4 da lista: O responsável pelo dano ambiental - aqui pode ser uma empresa ou pessoa física.
        (devolva a string "Não encontrado" caso não exista)
        Elemento 5 da lista: Categoria do elemento 4 - "Pessoa Física" ou "Pessoa Jurídica"
        (devolva a string "Não encontrado" caso não exista)
        Elemento 6 da lista: A data de ocorrência do impacto ambiental no formato DD/MM/AA
        (devolva a string "Não encontrado" caso não exista)
        Elemento 7 da lista: A extensão da área afetada pelo impacto ambiental
        (devolva a string "Não encontrado" caso não exista)
        Elemento 8 da lista: A unidade de medida do elemento 7
        (devolva a string "Não encontrado" caso não exista)
        Elemento 9 da lista: A metodologia de punição do responsável pelo impacto
        (devolva a string "Não encontrado" caso não exista)
        Elemento 10 da lista: O valor da multa que o responsável deve pagar pelas suas ações (em R$)
        (devolva a string "Não encontrado" caso não exista)
        Texto bruto: {texto_extraido}"""
generation_config_low_temp = genai.GenerationConfig(
    max_output_tokens=500,
    temperature=0.0,
)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
time.sleep(3)
resposta.text

"```\n[False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK AMBIENTAL - MARANHÃO S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado']\n```\n"

## Teste do Gemini para algumas amostras

In [43]:
respostas = {}

for i in range(0,10):
    response = requests.get(filtrados.iloc[i,3])

    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto_extraido = ""

    if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto_extraido += page.get_text()
    
    elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto_extraido = soup.get_text()

# Prompt
    prompt = f"""Você está recebendo um texto bruto de um documento sobre
        algum processo judiciário. O seu interesse principal é descobrir se este texto trata de um caso relacionado à 
        valoração de danos ambientais.
        Assim, a partir das informações nele contidas, devolva APENAS UMA LISTA (NÃO DEVOLVA O TEXTO) 
        com as seguintes informações em ordem:
        Elemento 0 da lista: Texto fala de um processo em que houve algum tipo de dano ambiental (True ou False)
        Elemento 1 da lista: Texto menciona uma georreferência do local afetado pelo dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 2 da lista: A UF do local ou seja "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará",
    "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará",
    "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro",
    "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima",
    "Santa Catarina", "São Paulo", "Sergipe", "Tocantins" ou variações (devolva a string
        "Não encontrado" caso não exista)
        Elemento 3 da lista: O município ou cidade do local do processo (devolva a string
        "Não encontrado" caso não exista). 
        Elemento 4 da lista: O responsável pelo dano ambiental - aqui pode ser uma empresa ou pessoa física.
        (devolva a string "Não encontrado" caso não exista)
        Elemento 5 da lista: Categoria do elemento 4 - "Pessoa Física" ou "Pessoa Jurídica"
        (devolva a string "Não encontrado" caso não exista)
        Elemento 6 da lista: A data de ocorrência do impacto ambiental no formato DD/MM/AA
        (devolva a string "Não encontrado" caso não exista)
        Elemento 7 da lista: A extensão da área afetada pelo impacto ambiental
        (devolva a string "Não encontrado" caso não exista)
        Elemento 8 da lista: A unidade de medida do elemento 7
        (devolva a string "Não encontrado" caso não exista)
        Elemento 9 da lista: A metodologia de punição do responsável pelo impacto
        (devolva a string "Não encontrado" caso não exista)
        Elemento 10 da lista: O valor da multa que o responsável deve pagar pelas suas ações (em R$)
        (devolva a string "Não encontrado" caso não exista)
        Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)

    resposta_eval = eval(resposta.text.strip("`\n "))

    # Insere info extra no início da lista (ex: nome ou id do processo)
    info_extra = filtrados.iloc[i,1]
    resposta_eval.insert(0, info_extra)

    respostas[filtrados.iloc[i,2]] = resposta_eval
    print(respostas)



{2643544408: [652049379, False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK Ambiental Maranhão S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado']}
{2643544408: [652049379, False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK Ambiental Maranhão S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado'], 2643544419: [652049379, False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK AMBIENTAL - MARANHÃO S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado']}
{2643544408: [652049379, False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK Ambiental Maranhão S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado'], 2643544419: [652049379, False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK AMBIENTAL - MARANHÃO S.A', 'Pessoa 

In [48]:
respostas_teste = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "georreferência",
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa"
]

In [49]:
display(respostas_teste)

,processoAnexoID,processo_ambiental,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa
2643544408,652049379,False,Não encontrado,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
2643544419,652049379,False,Não encontrado,Maranhão,Paço do Lumiar,BRK AMBIENTAL - MARANHÃO S.A,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
2643544426,652049379,True,"Alameda 5, nº 17, quadra 7, Residencial Paranã...",Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
590939495,612465137,True,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
590939496,612465137,True,Não encontrado,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
590939497,612465137,True,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
1186042345,582564055,False,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
2589045924,690025576,True,"Rodovia BR 494, Km 25, Bairro Várzea, municípi...",Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Não encontrado,"R$ 3.807.896,40"
2589045929,690025576,True,Não encontrado,Minas Gerais,Não encontrado,Carmense Comercial Ltda,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Não encontrado,Não encontrado
2589045772,690025576,True,"Rodovia BR 494, Km 25, Bairro Várzea, municípi...",Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,Não encontrado,Não encontrado,Não encontrado,Investigação Confirmatória,"2.447.933,38"


In [ ]:
percent_processos_amb = (respostas_teste.loc[respostas_teste["processo_ambiental"] == False, :].shape[0]/respostas_teste.shape[0])*100
print(f"Percentual de processos ambientais encontrados na amostra: {percent_processos_amb}")